In [1]:
from sliderule import icesat2, ipysliderule, io
import ipywidgets as widgets
import logging
import warnings
# autoreload
%load_ext autoreload
%autoreload 2
# turn off warnings for demo
warnings.filterwarnings('ignore')

In [2]:
# set the url for the sliderule service
# set the logging level
icesat2.init("icesat2sliderule.org", loglevel=logging.WARNING)

In [9]:
# display widgets for setting SlideRule parameters
SRwidgets = ipysliderule.widgets()
widgets.VBox([
    SRwidgets.asset,
    SRwidgets.release,
    #SRwidgets.classification,
    SRwidgets.surface_type,
    SRwidgets.confidence,
    SRwidgets.quality,
    SRwidgets.land_class,
    SRwidgets.yapc_knn,
    SRwidgets.yapc_win_h,
    SRwidgets.yapc_win_x,
    SRwidgets.yapc_min_ph,
    SRwidgets.yapc_weight,
    SRwidgets.length,
    SRwidgets.step,
    SRwidgets.iteration,
    SRwidgets.spread,
    SRwidgets.count,
    SRwidgets.window,
    SRwidgets.sigma,
])

In [10]:
widgets.VBox([
    SRwidgets.projection,
    SRwidgets.layers
])

In [13]:
# create ipyleaflet map in specified projection
m = ipysliderule.leaflet(SRwidgets.projection.value)
# install click handler callback
#m.add_selected_callback(SRwidgets.click_handler)
m.map

Map(center=[39, -108], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [14]:
m.add_layer(layers=SRwidgets.layers.value)

In [15]:
%%time
# sliderule asset and data release
asset = SRwidgets.asset.value
release = SRwidgets.release.value
# build sliderule parameters using latest values from widget
parms = SRwidgets.build()

# create an empty geodataframe
gdf = icesat2.__emptyframe()

# for each region of interest
for poly in m.regions:
    # add polygon from map to sliderule parameters
    parms["poly"] = poly 
    # make the request to the SlideRule (ATL06-SR) endpoint
    # and pass it the request parameters to request ATL06 Data
    gdf = gdf.append(icesat2.atl06p(parms, asset, version=release))

AttributeError: 'widgets' object has no attribute 'build'

In [ ]:
print(f'Returned {gdf.shape[0]} records')
gdf.head()

In [ ]:
widgets.VBox([
    SRwidgets.variable,
    SRwidgets.cmap,
    SRwidgets.reverse,
])

In [ ]:
%matplotlib inline
m.GeoData(gdf, column_name=SRwidgets.variable.value, cmap=SRwidgets.colormap)

In [ ]:
m.remove_layer(layers=m.layers)

In [ ]:
widgets.VBox([
    SRwidgets.plot_kind,
    SRwidgets.rgt,
    SRwidgets.ground_track,
    SRwidgets.cycle,
])


In [ ]:
%matplotlib widget
# default is to skip cycles with significant off-pointing
SRwidgets.plot(gdf, kind=SRwidgets.plot_kind.value, cycle_start=3, legend=True)